In [25]:
from transformation import transform_xml, convert_html_to_pdf

In [26]:
xml_dir = "/home/simon/scripts/xinvoices/test_instances/positive"
html_dir = "output/html"
pdf_dir = "output/pdf"
xsl_dir = "/home/simon/scripts/xinvoices/xsl"
# xsl_path = "/home/simon/scripts/xinvoices/xsl/simple.xsl"

# Define XSLT parameters if needed
params = {"lang": "de", "invoiceline-layout": "tabular"}

print("Transforming XML to HTML using Saxon/C and saxoncee...")
transform_xml(xml_dir, html_dir, xsl_dir)

print("Converting HTML to PDF using WeasyPrint...")
convert_html_to_pdf(html_dir, pdf_dir)

print("Transformation and PDF generation complete.")

2024-10-24 13:47:21,955 - INFO - Saxon/C Processor Version: SaxonC-HE 12.5 from Saxonica
2024-10-24 13:47:21,956 - INFO - Output directories ensured: output/html, output/html/../xr
2024-10-24 13:47:21,957 - INFO - Processing file: /home/simon/scripts/xinvoices/test_instances/positive/01.01a-INVOICE_uncefact_ubl_generated.xml
2024-10-24 13:47:21,957 - INFO - Compiling XSLT stylesheet: /home/simon/scripts/xinvoices/xsl/ubl-invoice-xr.xsl
2024-10-24 13:47:21,983 - INFO - Transforming '/home/simon/scripts/xinvoices/test_instances/positive/01.01a-INVOICE_uncefact_ubl_generated.xml' to 'output/html/../xr/01.01a-INVOICE_uncefact_ubl_generated-xr.xml' using XSLT.
2024-10-24 13:47:21,988 - INFO - Transformation for /home/simon/scripts/xinvoices/xsl/ubl-invoice-xr.xsl completed successfully.
2024-10-24 13:47:21,989 - INFO - Compiling XSLT stylesheet: /home/simon/scripts/xinvoices/xsl/xrechnung-html.xsl
2024-10-24 13:47:22,014 - INFO - Transforming 'output/html/../xr/01.01a-INVOICE_uncefact_ubl_g

Transforming XML to HTML using Saxon/C and saxoncee...


2024-10-24 13:47:22,168 - INFO - Transformation for /home/simon/scripts/xinvoices/xsl/xrechnung-html.xsl completed successfully.
2024-10-24 13:47:22,168 - INFO - Processing file: /home/simon/scripts/xinvoices/test_instances/positive/01.01_comprehensive_test_ubl.xml
2024-10-24 13:47:22,169 - INFO - Compiling XSLT stylesheet: /home/simon/scripts/xinvoices/xsl/ubl-invoice-xr.xsl
2024-10-24 13:47:22,260 - INFO - Transforming '/home/simon/scripts/xinvoices/test_instances/positive/01.01_comprehensive_test_ubl.xml' to 'output/html/../xr/01.01_comprehensive_test_ubl-xr.xml' using XSLT.
2024-10-24 13:47:22,276 - INFO - Transformation for /home/simon/scripts/xinvoices/xsl/ubl-invoice-xr.xsl completed successfully.
2024-10-24 13:47:22,277 - INFO - Compiling XSLT stylesheet: /home/simon/scripts/xinvoices/xsl/xrechnung-html.xsl
2024-10-24 13:47:22,297 - INFO - Transforming 'output/html/../xr/01.01_comprehensive_test_ubl-xr.xml' to 'output/html/01.01_comprehensive_test_ubl.html' using XSLT.
2024-10-

Converting HTML to PDF using WeasyPrint...


2024-10-24 13:47:23,010 - DEBUG - Reading 'maxp' table from disk
2024-10-24 13:47:23,011 - DEBUG - Decompiling 'maxp' table
2024-10-24 13:47:23,011 - DEBUG - Took 0.001s to load 'maxp'
2024-10-24 13:47:23,011 - DEBUG - Took 0.000s to prune 'maxp'
2024-10-24 13:47:23,012 - INFO - maxp pruned
2024-10-24 13:47:23,012 - DEBUG - Reading 'cmap' table from disk
2024-10-24 13:47:23,012 - DEBUG - Decompiling 'cmap' table
2024-10-24 13:47:23,013 - DEBUG - Reading 'post' table from disk
2024-10-24 13:47:23,013 - DEBUG - Decompiling 'post' table
2024-10-24 13:47:23,018 - DEBUG - Took 0.006s to load 'cmap'
2024-10-24 13:47:23,018 - DEBUG - Took 0.000s to prune 'cmap'
2024-10-24 13:47:23,019 - INFO - cmap pruned
2024-10-24 13:47:23,019 - INFO - fpgm dropped
2024-10-24 13:47:23,019 - INFO - prep dropped
2024-10-24 13:47:23,019 - INFO - cvt  dropped
2024-10-24 13:47:23,020 - INFO - kern dropped
2024-10-24 13:47:23,020 - DEBUG - Took 0.000s to load 'post'
2024-10-24 13:47:23,020 - DEBUG - Took 0.000s t

Transformation and PDF generation complete.
